In [3]:
from openai import OpenAI
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI as OpenAILC
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [2]:
llm = ChatOpenAI(
    openai_api_key=client.api_key,
    model_name="gpt-3.5-turbo"
)

Define a class for the pydantic custom model

In [4]:
class My_List(BaseModel):
    name: str = Field(description="")
    list_items: list = Field(description="")

Define a function that will generate the list base on input of a topic

In [13]:
def list_creator(topic):
    My_List.name="Name of the list of a {}".format(topic)
    My_List.list_items="A list of the {}".format(topic)

    parser = PydanticOutputParser(pydantic_object=My_List)

    human_text = "{instruction}\n{format_instructions}"
    human_prompt = HumanMessagePromptTemplate.from_template(human_text)

    chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

    prompt = chat_prompt.format_prompt(
        instruction="Create a list for {}".format(topic),
        format_instructions=parser.get_format_instructions()
    ).to_messages()

    reply = llm(prompt)

    return parser.parse(reply.content)

Define a function that will create content of a tweet, based on input of a topic

In [18]:
def tweet_creator(topic, tone):
    system_template = 'You are a social media expert'
    system_message = SystemMessagePromptTemplate.from_template(system_template)

    human_template = "Write a tweet about {topic}. The tone should be {tone}. The tweet should be no longer than 200 characters"
    human_message = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    prompt = chat_prompt.format_prompt(
        topic=topic,
        tone=tone
    ).to_messages()

    reply = llm(prompt)
    return reply.content

The main function that assembly the other functions

In [11]:
def main():
    topic=  input("Please enter a topic for a list:")
    result_list = list_creator(topic)
    print("-"*10)
    for i, result in enumerate(result_list.list_items, start=1):
        # use enumerate to iterate the list, with index number start from 1
        print("{}. {}".format(i, result))
    
    topic_number = int(input("Enter the topic number to make a tweet about it."))
    tone =  input("Enter the tone for the tweet.")
    tweet = tweet_creator(result_list.list_items[topic_number-1], tone)
    print("-"*10)
    print("The tweet: {}".format(tweet))

In [19]:
main()

----------
1. Presidential candidates
2. Vice Presidential candidates
3. Political parties
4. Key issues
5. Campaign strategies
6. Electoral map
7. Polling data
8. Debates schedule
9. Election results
----------
The tweet: "Something seems off about these election results... 🤔🗳️ #ElectionFraud #QuestionableWin #StayAlert"
